In [1]:
# Install libs
! pip install -qq unsloth==2025.4.1 vllm==0.7.3 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

## 1. Cài đặt thư viện và tải về bộ dữ liệu

In [2]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig, SFTConfig, SFTTrainer
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import wandb

from unsloth import FastLanguageModel
from peft import LoraConfig
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Preference Tuning with LLMs")
secret_value_1 = user_secrets.get_secret("push_to_hub")

wandb.login(key=secret_value_0)

2025-05-09 06:44:33.280899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746773073.492807      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746773073.559165      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/3424111535.py:8: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-09 06:44:54 __init__.py:207] Automatically detected platform cuda.


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moonlig73 (minhdeptrai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
CACHE_DIR = "/kaggle/working/"

dataset = load_dataset("thainq107/Vi-Alpaca-Preference", cache_dir=CACHE_DIR)
dataset_train = dataset["train"].select(range(50))
dataset_train[0]

README.md:   0%|          | 0.00/506 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/61.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65017 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'id': 'alpaca-7294',
 'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
 'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
 'rejected': 'Câu này không có lỗi ngữ pháp.'}

In [4]:
# ——— HẰNG SỐ & ENV VAR ———
MODEL_NAME      = "meta-llama/Llama-3.2-1B-Instruct"
MAX_SEQ          = 1024
LORA_RANK        = 8
SFT_OUTPUT_DIR   = "/kaggle/working/Llama-3.2-1B-Instruct-sft"
TARGET_MODULES   = [
    "q_proj","k_proj","v_proj","o_proj",
    "gate_proj","up_proj","down_proj",
]

# ——— LOAD MODEL + TOKENIZER + WRAP LoRA ———
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ,
    load_in_4bit=True,            # bật 4bit để đỡ OOM
    fast_inference=True,
    max_lora_rank=LORA_RANK,
    gpu_memory_utilization=0.8,
)

lora_model = FastLanguageModel.get_peft_model(
    base_model,
    r=LORA_RANK,
    target_modules=TARGET_MODULES,
    lora_alpha=LORA_RANK,
    use_gradient_checkpointing="unsloth",
)

# padding
if tokenizer.pad_token is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.pad_token    = tokenizer.eos_token

# ——— PEFT & SFT CONFIG ———
peft_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_RANK,
    target_modules=TARGET_MODULES,
    task_type="CAUSAL_LM",
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.1. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit with actual GPU utilization = 79.24%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 10.62 GB. Also swap space = 5 GB.
WARNING 05-09 06:45:06 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 05-09 06:45:19 config.py:549] This model supports multiple tasks: {'score', 'reward', 'generate', 'embed

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

INFO 05-09 06:46:46 model_runner.py:1562] Graph capturing finished in 40 secs, took 0.38 GiB
INFO 05-09 06:46:46 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 63.70 seconds


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.4.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## 2. Preprocessing

In [5]:
def formatting_prompt_with_chat_template(example):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": example["question"]},
        {"role": "assistant", "content": example["chosen"]},
    ]
    prompt = tokenizer.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=False
    )
    return prompt


## 3. Trainning SFT

In [6]:
import os
os.environ["UNSLOTH_RETURN_LOGITS"] = "1"

hyperparameters = {
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 1,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "fp16": True
}


SFT_OUTPUT_DIR = f"/kaggle/working/Llama-3.2-1B-Instruct-sft"

sft_config = SFTConfig(
    **{ **hyperparameters, "output_dir":
       SFT_OUTPUT_DIR , "max_seq_length": MAX_SEQ}
)
sft_trainer = SFTTrainer(
    model=lora_model,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset_train,
    formatting_func=formatting_prompt_with_chat_template
)

sft_trainer.train()

Applying formatting function to train dataset (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

Converting train dataset to ChatML (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 1 | Total steps = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 5,636,096/1,000,000,000 (0.56% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250509_064715-tfxbvxhx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run /kaggle/working/Llama-3.2-1B-Instruct-sft
wandb: ⭐️ View project at https://wandb.ai/minhdeptrai/huggingface
wa

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.8915942509969077, metrics={'train_runtime': 13.0588, 'train_samples_per_second': 3.829, 'train_steps_per_second': 0.23, 'total_flos': 47678183153664.0, 'train_loss': 2.8915942509969077})

## Push to hub

In [7]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(
    repo_id="TSunm/Llama-3.2-1B-Instruct-sft",  # repo_id = "username/repo_name"
    token=secret_value_1,
    repo_type="model",
    exist_ok=True
)

RepoUrl('https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='TSunm/Llama-3.2-1B-Instruct-sft')

In [8]:
sft_trainer.model.push_to_hub_merged(
    "TSunm/Llama-3.2-1B-Instruct-sft",
    token=secret_value_1
)
tokenizer.push_to_hub("TSunm/Llama-3.2-1B-Instruct-sft", token=secret_value_1)

Unsloth: You're not saving a tokenizer as well?
You can do it separately via `tokenizer.push_to_hub(...)`
Unsloth: You are pushing to hub in Kaggle environment.
To save memory, we shall move TSunm/Llama-3.2-1B-Instruct-sft to /tmp/Llama-3.2-1B-Instruct-sft
Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 13.16 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 12%|█▎        | 2/16 [00:00<00:00, 18.18it/s]
We will save to Disk and not RAM now.
100%|██████████| 16/16 [00:03<00:00,  4.22it/s]



Unsloth: Saving /tmp/Llama-3.2-1B-Instruct-sft/pytorch_model.bin...


README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-sft


No files have been modified since last commit. Skipping to prevent empty commit.
